# Image to Excel Converter

This Jupyter Notebook retrieves images from a local directory, compresses them into thumbnail size, and embeds them into an Excel spreadsheet. Additionally, it includes user-provided columns that can be customized.

## Features:
- Automatically fetches images from a specified directory.
- Converts images into thumbnails for efficient storage.
- Embeds images directly in an Excel spreadsheet.
- Allows users to input custom columns for additional information.

## Prerequisites:
- Install the required libraries if not already installed:
  ```bash
  pip install pillow pandas openpyxl
  ```
- Ensure images are stored in the `./images` directory or update the path accordingly.

In [ ]:
import os
import io
from PIL import Image
import pandas as pd
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage

# Define directory to fetch images
image_directory = "./images"  # Change this to your local directory

thumbnail_size = (540, 540)

def get_images_from_directory(directory):
    """Retrieve all image files from a directory."""
    image_files = [f for f in os.listdir(directory) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    return image_files

def create_excel_with_images(image_directory, custom_columns):
    """Creates an Excel workbook with images and user-provided custom columns."""
    
    # Get image files
    images = get_images_from_directory(image_directory)
    
    # Create DataFrame to store information
    data = []
    for img_file in images:
        data.append(["", img_file] + ["" for _ in custom_columns])  # Placeholder for custom columns
    
    df = pd.DataFrame(data, columns=['Image', 'Image Name'] + custom_columns)
    
    # Create Excel workbook and sheet
    wb = Workbook()
    ws = wb.active
    ws.title = "Images Data"
    
    # Add headers
    ws.append(df.columns.tolist())
    
    # Add image thumbnails and data
    for idx, row in df.iterrows():
        img_path = os.path.join(image_directory, row['Image Name'])
        if os.path.exists(img_path):
            img = Image.open(img_path)
            img.thumbnail(thumbnail_size)
            img_io = io.BytesIO()
            img.save(img_io, format='PNG')
            img_io.seek(0)
            excel_img = XLImage(img_io)
            cell = f'A{idx+2}'
            ws.add_image(excel_img, cell)
        
        ws[f'B{idx+2}'] = row['Image Name']
        
        for col_idx, col_name in enumerate(custom_columns, start=3):
            ws[f'{chr(65+col_idx)}{idx+2}'] = row[col_name]
    
    # Save workbook
    excel_path = "images_with_custom_columns.xlsx"
    wb.save(excel_path)
    print(f"Excel file saved as {excel_path}")

# Define customizable columns
custom_columns = ['Action', 'Direction']  # Modify as needed

# Run the function
create_excel_with_images(image_directory, custom_columns)